In [1]:
#import libraries 
import numpy as np
import pandas as pd

In [2]:
#Load the data
profileSrcJSON = pd.read_json('/Users/javi/Downloads/TinderProfiles.json')

In [3]:
#create a copy in order to work with it and not to damage our original dataSet
profileCopy = profileSrcJSON.copy()

In [4]:
#Drop off "__v" column as it has only 0 values
del profileCopy["__v"]

In [5]:
#define the dataframe 
preDf = profileCopy

In [6]:
preDf.head(3)

,_id,appOpens,conversations,conversationsMeta,matches,messages,messagesReceived,messagesSent,swipeLikes,swipePasses,swipes,user,userId
0,00b74e27ad1cbb2ded8e907fcc49eaaf,"{'2016-01-02': 26, '2016-01-13': 10, '2016-01-...","[{'match_id': 'Match 739', 'messages': [{'to':...","{'nrOfConversations': 739, 'longestConversatio...","{'2016-01-02': 10, '2016-01-13': 5, '2016-01-1...","{'sent': {'2016-01-02': 11, '2016-01-13': 2, '...","{'2016-01-02': 12, '2016-01-13': 3, '2016-01-1...","{'2016-01-02': 11, '2016-01-13': 2, '2016-01-1...","{'2016-01-02': 50, '2016-01-13': 70, '2016-01-...","{'2016-01-02': 14, '2016-01-13': 93, '2016-01-...","{'likes': {'2016-01-02': 50, '2016-01-13': 70,...","{'birthDate': '1976-01-01T00:00:00.000Z', 'age...",00b74e27ad1cbb2ded8e907fcc49eaaf
1,024610702baf540af5637873cd1534e9,"{'2016-07-12': 20, '2016-07-13': 18, '2016-07-...","[{'match_id': 'Match 464', 'messages': [{'to':...","{'nrOfConversations': 464, 'longestConversatio...","{'2016-07-12': 1, '2016-07-13': 0, '2016-07-14...","{'sent': {'2016-07-12': 0, '2016-07-13': 0, '2...","{'2016-07-12': 0, '2016-07-13': 0, '2016-07-14...","{'2016-07-12': 0, '2016-07-13': 0, '2016-07-14...","{'2016-07-12': 0, '2016-07-13': 221, '2016-07-...","{'2016-07-12': 0, '2016-07-13': 61, '2016-07-1...","{'likes': {'2016-07-12': 0, '2016-07-13': 221,...","{'birthDate': '1997-07-04T00:00:00.000Z', 'age...",024610702baf540af5637873cd1534e9
2,0a5e3dd8489fe67485ddb7d6adb26ebd,"{'2019-07-01': 61, '2019-07-02': 154, '2019-07...","[{'match_id': 'Match 303', 'messages': [{'to':...","{'nrOfConversations': 303, 'longestConversatio...","{'2019-07-01': 13, '2019-07-02': 12, '2019-07-...","{'sent': {'2019-07-01': 12, '2019-07-02': 66, ...","{'2019-07-01': 6, '2019-07-02': 70, '2019-07-0...","{'2019-07-01': 12, '2019-07-02': 66, '2019-07-...","{'2019-07-01': 87, '2019-07-02': 39, '2019-07-...","{'2019-07-01': 30, '2019-07-02': 12, '2019-07-...","{'likes': {'2019-07-01': 87, '2019-07-02': 39,...","{'birthDate': '1998-02-06T00:00:00.000Z', 'age...",0a5e3dd8489fe67485ddb7d6adb26ebd


### Each profile has an ID number which we use to check how many users are in the dataset. We also verify that there aren't any Duplicate Accounts in this dataset before continuing on with our research!

In [7]:
userid = []
for uniqueid in preDf.userId:
    userid.append(uniqueid[-6:])

In [8]:
totalUserid = userid[:1180]

### We want to see how many times each user opened the app in our dataset, and also what their average daily number was.


In [9]:
import sys 

#check how many people open the app (daily)
personOpeners = []
for openers in preDf.appOpens:
    if len(openers) == 0:
        continue
    maxDate = ""
    maxOpens = 0
    minDate = ""
    minOpens = sys.maxsize
    for date in openers:
        if (openers[date] > maxOpens):
            maxOpens = openers[date]
            maxDate = date

        if (openers[date] < minOpens):
            minOpens = openers[date]
            minDate = date

    personOpeners.append(minOpens)
    # append the vals to a list

counter = 0
for j in personOpeners:
    if j >= 1:
        counter += 1 

print(f'there are {counter} people who open the app every day')


there are 290 people who open the app every day


In [10]:
#import the langdetect in order to detect the lenguaje of the message
#We will focus on English, Spanish and Japanese 
from langdetect import detect

### Since the messages are in a JSON structure, we have to take those out in order to do our language classification and run the sentiment analysis

In [11]:
allMessages = []
numberConversations = []
for conversation in preDf.conversations:
    messages = []
    numberMessages = 0
    for match in conversation:
        for message in match['messages']:
            if 'message' in message.keys():
                messages.append(message['message'])
                numberMessages += 1
    # messages = messages[:5]
    allMessages.append(messages)
    numberConversations.append(numberMessages)
len(allMessages)

1209

In [12]:
totalMessages = allMessages[:1180]
totalnumberConversations = numberConversations[:1180]

In [13]:
# identify the lenguaje of the user
# extract the first line of each conversation
alllengu = []
for conversation in preDf.conversations:
    for match in conversation:
        lengu = []
        for message in match['messages']:
            if 'message' in message.keys():
                    try:
                        language = detect(message['message'])
                        lengu.append(language) 
                    except:
                        1 
        lengu = lengu[:1]
        alllengu.append(lengu)
        break

In [14]:
len(alllengu)

1180

In [15]:
#list of numbers, each number is a user 
totalMatches = []

for dfMatches in preDf.matches:
    #variable name 
    totalMatchesUser = 0
    for MatchDate in dfMatches:
        totalMatchesUser += dfMatches[MatchDate]

    totalMatches.append(totalMatchesUser)


In [16]:
totalMatches = totalMatches[:1180]

In [17]:
#total swipes right per use
totalSwipesRight = [] 
for dfSwipeLikes in preDf.swipeLikes:
    countSwipesRight = 0
    for swipeRight in dfSwipeLikes:
        countSwipesRight += dfSwipeLikes[swipeRight]
    totalSwipesRight.append(countSwipesRight)

In [18]:
totalSwipesRight = totalSwipesRight[:1180]

In [28]:
#total swipes left per user
totalSwipesLeft = []
for dfSwipePasses in preDf.swipePasses:
    countSwipesLeft = 0
    for swipeLeft in dfSwipePasses:
        countSwipesLeft += dfSwipePasses[swipeLeft]
    totalSwipesLeft.append(countSwipesLeft)

In [29]:
totalSwipesLeft = totalSwipesLeft[:1180]

In [30]:
#import library for know the user age 
from datetime import date
from datetime import datetime

#Take user anonymous data and break down the dictionary format
allBirthDates=[]

#Users date of birth
def age(birthdate):
    today = date.today()
    age = today.year - birthdate.year #- ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

for person in preDf.user:
    birthdateStr = person['birthDate'][:10]
    birthdate = datetime.strptime(birthdateStr, '%Y-%m-%d')
    userAge = age(birthdate)
    allBirthDates.append(userAge)

In [31]:
totalAllBirthDates = allBirthDates[:1180]

### CREATE THE DATASET WITH DATA WE HAVE CLEANED

In [48]:
#We insert the unique user id
df = pd.DataFrame(totalUserid, columns=['user'])
df.head(3)

,user
0,49eaaf
1,1534e9
2,b26ebd


In [49]:
#check if there is user with Null value in their id
pd.isnull(df.loc[1, 'user'])

False

In [50]:
#Merge app_opens data with df
df['Messages'] = totalMessages
df['Language'] = alllengu
df['SwipesRight'] = totalSwipesRight
df['SwipesLeft'] = totalSwipesLeft
df['Matches'] = totalMatches
df['Num Of Messages'] = totalnumberConversations
df['Age'] = totalAllBirthDates

In [52]:
df.head()

,user,Messages,Language,SwipesRight,SwipesLeft,Matches,Num Of Messages,Age
0,49eaaf,[],[],23341,24229,3408,0,46
1,1534e9,"[Hello again, so now that we matched again, wh...",[en],45068,49559,888,2086,25
2,b26ebd,"[It&rsquo;s me, the chicken nuggets, Exactly w...",[en],6679,5500,548,1289,24
3,ffbf58,[I&apos;m worried about you. Based on your hik...,[en],6724,7899,94,371,24
4,8a5cc1,[],[],60169,96673,1905,0,26


In [53]:
len(df.index)

1180

In [54]:
#drop users with empty list
df = df[~df.Messages.str.len().eq(0)]
df.head()

,user,Messages,Language,SwipesRight,SwipesLeft,Matches,Num Of Messages,Age
1,1534e9,"[Hello again, so now that we matched again, wh...",[en],45068,49559,888,2086,25
2,b26ebd,"[It&rsquo;s me, the chicken nuggets, Exactly w...",[en],6679,5500,548,1289,24
3,ffbf58,[I&apos;m worried about you. Based on your hik...,[en],6724,7899,94,371,24
5,d6e87e,"[Holaaa Andrea! Igual bonito inicio de semana,...",[it],22552,5477,203,2768,28
8,a0a92f,"[Terve😊, Helsingissä asun mut nyt frendin möki...",[],4532,115619,1781,5383,30


In [55]:
len(df.index)

1138

In [56]:
#drop users with 0  SwipesRight or 0 SwipesLeft
df = df.loc[~((df['SwipesRight'] == 0) | (df['SwipesLeft'] == 0) | (df['Num Of Messages'] == 0))]
df

,user,Messages,Language,SwipesRight,SwipesLeft,Matches,Num Of Messages,Age
1,1534e9,"[Hello again, so now that we matched again, wh...",[en],45068,49559,888,2086,25
2,b26ebd,"[It&rsquo;s me, the chicken nuggets, Exactly w...",[en],6679,5500,548,1289,24
3,ffbf58,[I&apos;m worried about you. Based on your hik...,[en],6724,7899,94,371,24
5,d6e87e,"[Holaaa Andrea! Igual bonito inicio de semana,...",[it],22552,5477,203,2768,28
8,a0a92f,"[Terve😊, Helsingissä asun mut nyt frendin möki...",[],4532,115619,1781,5383,30
...,...,...,...,...,...,...,...,...
1173,be6a5a,"[I am great, whats up?, Yesd, Titanic xD, I he...",[nl],581,2586,4,121,24
1174,fccfbe,"[You&rsquo;re kinda cute 🙈, Where are you now?...",[da],89628,17792,5393,21339,34
1175,86d422,"[Hi, Yes, Stats?, how&apos;s your weekend been...",[pl],2207,24655,524,2886,22
1176,fb50c2,[Hey! How&apos;s the covid life treating you? ...,[],33391,7014,306,1235,26


In [57]:
len(df.index)

934

In [58]:
#drop users with empty list
df = df[~df.Language.str.len().eq(0)]

In [59]:
df.head()

,user,Messages,Language,SwipesRight,SwipesLeft,Matches,Num Of Messages,Age
1,1534e9,"[Hello again, so now that we matched again, wh...",[en],45068,49559,888,2086,25
2,b26ebd,"[It&rsquo;s me, the chicken nuggets, Exactly w...",[en],6679,5500,548,1289,24
3,ffbf58,[I&apos;m worried about you. Based on your hik...,[en],6724,7899,94,371,24
5,d6e87e,"[Holaaa Andrea! Igual bonito inicio de semana,...",[it],22552,5477,203,2768,28
9,90b7d6,"[hey, nice pics ;&rpar;, well so how are you d...",[so],52133,45064,5075,7567,27


In [60]:
len(df.index)

792

In [61]:
#explode the colum messages so we can do the sentiment analysis 
df = df.explode('Messages')

In [45]:
df.head()

,user,Messages,Lenguage,SwipesRight,SwipesLeft,Matches,Num Of Messages,Age
1,1534e9,"Hello again, so now that we matched again, wha...",[en],45068,49559,888,2086,25
1,1534e9,"Most of the girls on this app scare me, but fo...",[en],45068,49559,888,2086,25
1,1534e9,"Dear diary, cute girl vanished... should I sen...",[en],45068,49559,888,2086,25
1,1534e9,"Most of the girls on this app scare me, but fo...",[en],45068,49559,888,2086,25
1,1534e9,"Well not many people have photos of dogs, and ...",[en],45068,49559,888,2086,25


In [62]:
#export to csv
df.to_csv('language.csv', encoding='utf-8')

In [ ]:
#Counter how many males 
countM = 0
for user in preDf.user:
    if user['gender'] == 'M':
        countM += 1

In [ ]:
#Counter how many males 
countF = 0
for user in preDf.user:
    if user['gender'] == 'F':
        countF += 1

In [ ]:
#Counter people who are interested in 'M'
counterM = 0
for user in preDf.user:
    if user["interestedIn"] == "M":
        counterM += 1
print(f"The number of people interested in 'M' are:  {counterM}")

The number of people interested in 'M' are:  134


In [ ]:
#Counter people who are interested in 'F'
counterF = 0
for user in preDf.user:
    if user["interestedIn"] == "F":
        counterF += 1
print(f"The number of people interested in 'F' are:  {counterF}")

The number of people interested in 'F' are:  1017


In [ ]:
#Counter people who are interested in 'M and F'
counterBi = 0
for user in preDf.user:
    if user["interestedIn"] != "F" and user["interestedIn"] != "M":
        counterBi += 1
print(f"The number of people interested in 'M and F' are:  {counterBi}")

The number of people interested in 'M and F' are:  58
